<a href="https://colab.research.google.com/github/dibend/Colab/blob/main/Investing_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install feedparser gradio

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.6/318.6 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━

In [ ]:
# Import libraries
import feedparser
import gradio as gr
from datetime import datetime

# Define the RSS feed URLs and their categories
rss_feeds = {
    "Top Stories": "https://feeds.content.dowjones.io/public/rss/mw_topstories",
    "Real-Time Headlines": "https://feeds.content.dowjones.io/public/rss/mw_realtimeheadlines",
    "Bulletins": "http://feeds.marketwatch.com/marketwatch/bulletins",
    "Market Pulse": "https://feeds.content.dowjones.io/public/rss/mw_marketpulse",
    "Nasdaq Original": "https://www.nasdaq.com/feed/nasdaq-original/rss.xml",
    "Commodities": "https://www.nasdaq.com/feed/rssoutbound?category=Commodities",
    "Cryptocurrencies": "https://www.nasdaq.com/feed/rssoutbound?category=Cryptocurrencies",
    "Dividends": "https://www.nasdaq.com/feed/rssoutbound?category=Dividends",
    "Earnings": "https://www.nasdaq.com/feed/rssoutbound?category=Earnings",
    "ETFs": "https://www.nasdaq.com/feed/rssoutbound?category=ETFs",
    "IPOs": "https://www.nasdaq.com/feed/rssoutbound?category=IPOs",
    "Markets": "https://www.nasdaq.com/feed/rssoutbound?category=Markets",
    "Options": "https://www.nasdaq.com/feed/rssoutbound?category=Options",
    "Stocks": "https://www.nasdaq.com/feed/rssoutbound?category=Stocks"
}

# Set User-Agent to mimic an iPhone
user_agent = "Mozilla/5.0 (iPhone; CPU iPhone OS 14_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1"

# Function to parse a single RSS feed and return the latest headlines
def parse_feed(url):
    headers = {'User-Agent': user_agent}
    feed = feedparser.parse(url, request_headers=headers)
    return feed.entries

# Function to get the latest headlines for each category
def get_latest_headlines():
    headlines = {}
    all_entries = []
    for category, url in rss_feeds.items():
        try:
            print(f"Parsing feed: {url}")
            entries = parse_feed(url)
            if entries:
                headlines[category] = entries  # Get all entries for each category
                all_entries.extend(entries)  # Add to all entries list
                print(f"Successfully parsed {len(entries)} entries from feed: {url}")
            else:
                print(f"No entries found in feed: {url}")
        except Exception as e:
            print(f"Error parsing feed {url}: {e}")
            headlines[category] = []
    # Sort all entries by published date
    sorted_all_entries = sorted(all_entries, key=lambda x: x.published_parsed, reverse=True)
    headlines["All"] = sorted_all_entries  # Include all entries sorted by recency in the All tab
    return headlines

# Function to format the headlines for display
def format_headlines():
    headlines = get_latest_headlines()
    formatted_headlines = {category: "" for category in rss_feeds.keys()}
    formatted_headlines["All"] = ""
    for category, entries in headlines.items():
        for entry in entries:
            entry_info = f"Title: {entry.title}\nLink: {entry.link}\nPublished: {entry.published}\n\n"
            formatted_headlines[category] += entry_info
            formatted_headlines["All"] += entry_info
    return formatted_headlines

# Function to create Gradio interface with tabs for each category
def create_interface():
    with gr.Blocks() as iface:
        with gr.Row():
            refresh_button = gr.Button("Refresh Feeds")

        tabs = gr.Tabs()
        with tabs:
            headlines = format_headlines()
            for category in headlines.keys():
                with gr.TabItem(category):
                    gr.Markdown(headlines[category])

        refresh_button.click(fn=create_interface, inputs=[], outputs=tabs)
    return iface

# Create the Gradio interface
iface = create_interface()

# Launch the Gradio interface
iface.launch(share=True, debug=True)

Parsing feed: https://feeds.content.dowjones.io/public/rss/mw_topstories
Successfully parsed 10 entries from feed: https://feeds.content.dowjones.io/public/rss/mw_topstories
Parsing feed: https://feeds.content.dowjones.io/public/rss/mw_realtimeheadlines
Successfully parsed 10 entries from feed: https://feeds.content.dowjones.io/public/rss/mw_realtimeheadlines
Parsing feed: http://feeds.marketwatch.com/marketwatch/bulletins
Successfully parsed 10 entries from feed: http://feeds.marketwatch.com/marketwatch/bulletins
Parsing feed: https://feeds.content.dowjones.io/public/rss/mw_marketpulse
Successfully parsed 30 entries from feed: https://feeds.content.dowjones.io/public/rss/mw_marketpulse
Parsing feed: https://www.nasdaq.com/feed/nasdaq-original/rss.xml
Successfully parsed 15 entries from feed: https://www.nasdaq.com/feed/nasdaq-original/rss.xml
Parsing feed: https://www.nasdaq.com/feed/rssoutbound?category=Commodities
Successfully parsed 15 entries from feed: https://www.nasdaq.com/feed